# Attention vs ARI Comparison
In this notebook we aim to compare the SAS trajectory patterns with the vowel clustering results. 

In [32]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# from C_0B_eval import *
from C_0X_defs import *
from C_0Y_evaldefs import *
from C_0Z_evaldefsAppendix import *
from scipy.stats import sem, ttest_ind
from misc_recorder import ListRecorder
import pandas as pd

In [33]:
ts = "0915163757" # this timestamp does not contain run number
train_name = "C_0Tg"
res_save_dir = os.path.join(model_save_, f"eval-{train_name}-{ts}")
swapped_res_save_dir = os.path.join(model_save_, f"evalswapped-{train_name}-{ts}")

In [ ]:
super_data_collector = {}   # 存放所有数据

In [ ]:
# 每個模型哪些run可以使用, 默認全部一樣
runs_dict = {
    "recon4-phi": [1, 2, 3, 4, 5], 
    "recon8-phi": [1, 2, 3, 4, 5], 
    "recon16-phi": [1, 2, 3, 4, 5], 
    "recon32-phi": [1, 2, 3, 4, 5], 
    "recon48-phi": [1, 2, 3, 4, 5], 
    "recon64-phi": [1, 2, 3, 4, 5], 
    "recon96-phi": [1, 2, 3, 4, 5], 
}

model_condition = "b"

In [ ]:
for model_type in ['recon4-phi', 'recon8-phi', 'recon16-phi', 'recon32-phi', 'recon48-phi', 'recon64-phi', 'recon96-phi']: 
    hiddim = int(model_type.split('-')[0].split('recon')[1])
    super_data_collector[hiddim] = {} # 每個model添加數據 (hiddim)

    # 基於采用哪些run生成strseq作記錄
    learned_runs = runs_dict[model_type]
    string_learned_runs = [str(num) for num in learned_runs]
    strseq_learned_runs = "".join(string_learned_runs)

    # 該model的所有run的數據
    every_attns = []
    every_sepframes1 = []
    every_sepframes2 = []
    every_phi_types = []

    model_condition_dir = os.path.join(res_save_dir, model_type, model_condition)

    for epoch in range(0, 101): 
        # 對於每個epoch, 收集每個run的數據
        cat_attns = []
        cat_sepframes1 = []
        cat_sepframes2 = []
        cat_phi_types = []
        for run_number in learned_runs:
            this_model_condition_dir = os.path.join(model_condition_dir, f"{run_number}")
            allres = read_result_at(this_model_condition_dir, epoch)
            cat_phi_types += allres["phi-type"]
            cat_attns += allres["attn"]
            cat_sepframes1 += allres["sep-frame1"]
            cat_sepframes2 += allres["sep-frame2"]

        every_attns.append(cat_attns)
        every_sepframes1.append(cat_sepframes1)
        every_sepframes2.append(cat_sepframes2)
        every_phi_types.append(cat_phi_types)    

    attnres = collect_attention_epoch_trajectory(every_phi_types, every_attns, every_sepframes1, every_sepframes2, os.path.join(res_save_dir, f"attnepochtraj-at-all-{model_type}-{model_condition}-{strseq_learned_runs}-10pc.png"), 
                                    conditionlist=["T", "ST"])
    
    super_data_collector[hiddim]["attnres"] = attnres